<a href="https://colab.research.google.com/github/r1marcus/TraintheTrainer-Daten/blob/main/01_Daten_load_Data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Modul Daten

Dieses Jupyter Notebook soll einen Überblick über die Aufbereitung, Bereinigung und Visualisierung von Daten vermitteln. 

Für diesen Zweck wurde die Datenbank Gas Sensor Array Drift verwendet, heruntergeladen und vorher für diesen Zweck manipuliert (Gas- und Konzentrationsklassenlaben wurden zu einem Label zusammen geführt und dabei drei verschiedene Konzentrationsstufen definiert: low, medium und high). <br>
Der vollständige Datensatz kann auf <br>

http://archive.ics.uci.edu/ml/datasets/Gas+Sensor+Array+Drift+Dataset+at+Different+Concentrations

heruntergeladen werden. Außerdem werden verschiedene Informationen zu den Daten bereitgestellt. <br>

Der Datensatz enthält 13.910 Messungen von 16 chemischen Sensoren, welche abwechselnd sechs verschiedene Gase (Ethanol, Ethen, Ammoniak, Acetaldehyd, Aceton und Toluol) in verschiedenen Konzentrationen untersuchen. <br>
Der Datensatz wurde zwischen Januar 2008 und Februar 2011 (36 Monaten) erhoben. <br>
Jede Messung liefert anhand der 16 vorhandenen Sensoren eine 16-Kanal-Zeitreihe. <br>
Es werden zwei Hauptfeatures in diesem Datensatz betrachtet: <br>
$\rightarrow~$ (i) das stationäre Feature (steady-state) bezeichnet als `DR`, definiert als die maximale Widerstandsänderung in Bezug auf eine Basis, sowie die normalisierte Version davon (`|DR|`). <br>
$\rightarrow~$ (ii) ein Ansammlung an Features welche die Sensordynamik der gesamten Messung wiederspiegelt (`EMAi` und `EMAd` für verschiedene $\alpha$-Werte). <br>

Im folgendem Beispiel werden wir uns nur mit dem `DR` Feature auseinandersetzen. Da jeder der 16 Sensoren diesen Messwert liefert, sind auch 16 verschiedene Messwerte von `DR` vorhanden (`DR_1` des ersten Sensors, `DR_2`, des zweiten Sensorns und wo weiter). 

## Laden des Pandas DataFrames

Als erstes müssen alle Beispieldatensätze geladen werden. Dieser liegen in 10 verschiedene csv Dateien vor (batch0.csv bis batch9.csv). Anschließend werden diese zu einem gemeinsamen Datensatz zusammengeführt.

In [13]:
!rm -r /content/TraintheTrainer-Daten
!git clone https://github.com/r1marcus/TraintheTrainer-Daten.git
!echo $CWD
!cp -rf /content/TraintheTrainer-Daten/* /content/
!rm -r /content/TraintheTrainer-Daten

Cloning into 'TraintheTrainer-Daten'...
remote: Enumerating objects: 43, done.
remote: Counting objects: 100% (43/43), done.
remote: Compressing objects: 100% (41/41), done.
remote: Total 43 (delta 13), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (43/43), done.



In [2]:
import scripts.load



In [3]:
df_modified, df_original,all_files_mod,all_files_original = scripts.load.load()


Download dataset from URL: http://archive.ics.uci.edu/ml/machine-learning-databases/00270/
Extract files to : /content/input/dataset_dat//driftdataset.zip
Convert dat files to csv
445 lines have been written for batch 1
1244 lines have been written for batch 2
1586 lines have been written for batch 3
161 lines have been written for batch 4
197 lines have been written for batch 5
2300 lines have been written for batch 6
3613 lines have been written for batch 7
294 lines have been written for batch 8
470 lines have been written for batch 9
3600 lines have been written for batch 10
13910 lines have been written in total
call load_svmlight_file for batch1.csv
call load_svmlight_file for batch2.csv
call load_svmlight_file for batch3.csv
call load_svmlight_file for batch4.csv
call load_svmlight_file for batch5.csv
call load_svmlight_file for batch6.csv
call load_svmlight_file for batch7.csv
call load_svmlight_file for batch8.csv
call load_svmlight_file for batch9.csv
call load_svmlight_file 

## Ein kurzer Blick auf die Datenstruktur

Um ein Gefühl für die Datenstruktur zu erhalten bietet Pandas die folgenden Methoden an: <br>

`head(n)` $\longrightarrow~$ Gibt die $n$ ersten Zeilen (default $n$ = 5) aus. Nützlich um schnell zu testen, ob das Objekt im richtigen Datentyp vorliegt. <br>
`info()` $\longrightarrow~$ Gibt eine übersichtliche Zusammenfassung des DataFrames aus, einschließlich des Index dtypes, des Spalten dtypes, Nicht-Null-Werte sowie den Speicherverbrauch. <br>
`describe()` $\longrightarrow~$ Erstellt deskriptive Statistiken, welche die zentrale Tendenz, Verteilung und Form der Verteilung eines Datensatzes unter Ausschluss von NaN-Werten zusammenfasst. <br>
`value_count()` $\longrightarrow~$ Gibt die Anzahl eindeutiger Werte in absteigender Reihenfolge wieder. Muss an einem Feature aufgerufen werden.

Werfen wir als erstes ein Blick auf die `head()` Methode:

In [4]:
df_modified.head()

,target,DR_1,|DR|_1,EMAi0.001_1,EMAi0.01_1,EMAi0.1_1,EMAd0.001_1,EMAd0.01_1,EMAd0.1_1,DR_2,|DR|_2,EMAi0.001_2,EMAi0.01_2,EMAi0.1_2,EMAd0.001_2,EMAd0.01_2,EMAd0.1_2,DR_3,|DR|_3,EMAi0.001_3,EMAi0.01_3,EMAi0.1_3,EMAd0.001_3,EMAd0.01_3,EMAd0.1_3,DR_4,|DR|_4,EMAi0.001_4,EMAi0.01_4,EMAi0.1_4,EMAd0.001_4,EMAd0.01_4,EMAd0.1_4,DR_5,|DR|_5,EMAi0.001_5,EMAi0.01_5,EMAi0.1_5,EMAd0.001_5,EMAd0.01_5,...,DR_12,|DR|_12,EMAi0.001_12,EMAi0.01_12,EMAi0.1_12,EMAd0.001_12,EMAd0.01_12,EMAd0.1_12,DR_13,|DR|_13,EMAi0.001_13,EMAi0.01_13,EMAi0.1_13,EMAd0.001_13,EMAd0.01_13,EMAd0.1_13,DR_14,|DR|_14,EMAi0.001_14,EMAi0.01_14,EMAi0.1_14,EMAd0.001_14,EMAd0.01_14,EMAd0.1_14,DR_15,|DR|_15,EMAi0.001_15,EMAi0.01_15,EMAi0.1_15,EMAd0.001_15,EMAd0.01_15,EMAd0.1_15,DR_16,|DR|_16,EMAi0.001_16,EMAi0.01_16,EMAi0.1_16,EMAd0.001_16,EMAd0.01_16,EMAd0.1_16
0,Aceton medium,14273.577305,3.024320,2.938083,2.726658,9.160230,-1.088828,-2.553627,-10.008018,1616.000362,13.890712,0.065627,1.832977,3.794835,-0.626880,-2.216111,-3.727923,6427.087984,2.809918,1.644583,2.532359,6.366296,-1.083000,-1.859664,-7.400924,6398.330797,2.600863,1.582393,2.253560,6.377379,-0.983363,-1.951279,-7.636000,1664.318035,1.712735,0.479783,0.937948,3.270469,-0.270650,-0.684654,...,5754.598582,2.540110,1.363714,2.118585,3.919899,-0.891826,-1.491210,-4.799816,2309.367190,1.854152,0.644808,1.406715,3.644021,-0.395535,-0.659693,-3.172405,2480.520182,1.839672,0.601987,1.240567,3.631822,-0.365433,-0.663033,-2.662408,9574.647480,2.868638,2.051181,3.511200,6.889017,-1.414937,-2.603527,-7.025044,9204.196428,2.878063,1.929711,2.851631,5.784202,-1.375681,-2.058877,-6.341305
1,Acetaldehyd medium,19183.742717,4.357248,5.351283,6.450441,10.872395,-3.153488,-4.326323,-13.631954,32779.114891,11.340203,9.239101,13.293717,22.242999,-6.458544,-21.354226,-90.474834,13501.751635,3.877614,4.320766,7.807666,13.339435,-2.905348,-4.666630,-13.381673,13138.149759,3.729742,4.354278,8.237309,12.779460,-2.891614,-4.771663,-13.973791,3533.332016,2.520985,1.387820,3.959387,7.226592,-0.875622,-1.673741,...,12762.294711,3.833807,3.810493,6.410521,8.636354,-2.640514,-3.715027,-8.943377,4484.238742,2.534278,1.706882,5.160090,7.611733,-1.008812,-1.742027,-3.795255,4361.580382,2.459243,1.677987,4.050821,6.133668,-0.911240,-1.473621,-3.553569,20587.874047,4.577326,6.864667,13.242215,17.784286,-4.804865,-7.447718,-14.405438,18558.609048,4.406637,6.112619,10.939485,15.560763,-4.029051,-6.189030,-12.722531
2,Aceton medium,10348.227144,2.892709,2.513980,3.315183,7.719371,-2.329644,-2.840900,-10.222801,33021.895913,1.298981,7.282501,9.832498,15.060181,-4.554793,-6.859986,-22.271768,6916.928390,2.625112,1.695209,2.617707,6.490257,-1.130966,-1.798130,-8.096225,6010.516346,2.438325,1.593136,2.925605,6.128234,-1.091027,-2.012148,-7.936757,1684.378924,1.806566,0.562898,1.014501,4.406864,-0.328156,-0.810867,...,6227.614688,2.477222,1.488496,1.766931,4.541895,-0.940447,-1.418934,-4.318310,2488.615782,1.940033,0.729604,1.193182,3.213898,-0.429049,-0.785777,-3.177144,2746.199527,1.898727,0.673266,1.264984,3.253012,-0.387956,-0.692665,-3.078034,10134.424917,2.812022,2.523820,3.231065,6.992326,-1.805799,-2.744733,-7.404107,8615.469855,2.798972,1.906785,2.826327,5.702202,-1.371283,-2.398717,-6.272815
3,Acetaldehyd medium,16016.935601,3.524554,3.446228,3.858046,8.080305,-2.383137,-4.299175,-10.630594,28158.243096,-11.516644,8.739770,14.167981,17.563641,-10.842535,-43.908037,-100.421353,13454.689312,3.805564,4.173674,7.725053,11.070560,-2.828587,-4.722814,-13.844055,13156.418378,3.593533,4.335864,8.435083,13.599997,-2.814152,-4.610665,-13.685235,3592.771859,2.445125,1.405718,4.128753,7.337663,-0.824154,-1.655907,...,12418.923323,3.701550,3.659455,6.122430,9.013708,-2.664477,-3.628107,-7.414914,4443.382391,2.431713,1.702189,4.895521,7.343933,-1.003530,-1.731181,-4.002193,4355.735511,2.435027,1.597393,4.010580,6.137122,-0.896229,-1.505073,-3.702279,20099.996283,4.385018,6.986860,13.259128,16.647219,-4.856891,-7.048007,-16.391256,17823.751293,4.270726,5.988227,

Die erste Spalte verät uns das Klassenlabel. Die erste Instanz ist demnach eine Messung am Gas Ethanol bei niedriger Konzentration. Alle weitere Spalten beinhalten Daten die zu diesem Zeitpunkt bzw. Messpunkt erhoben worden sind. <br>

Rufen wir die `info()` Methode an den ersten fünf Spalten auf.

In [5]:
df_modified.iloc[:,:5].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13910 entries, 0 to 13909
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   target       13910 non-null  object 
 1   DR_1         13892 non-null  float64
 2   |DR|_1       13891 non-null  float64
 3   EMAi0.001_1  13896 non-null  float64
 4   EMAi0.01_1   13889 non-null  float64
dtypes: float64(4), object(1)
memory usage: 543.5+ KB


Für Pandas Indexing und Slicing siehe https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html. <br>

`info()` listet uns die verschiedene dtypes auf: vier float64 und eins vom Typ object. Anhand der unterschiedlichen Instanzen (total: 13910, DR_1: 13892, ...) erkennen wir, dass der Datensatz unvollständig ist (NaNs können ein Grund dafür sein). <br>

Werfen wir als nächstes ein Blick auf die `describe()` Methode.

In [6]:
df_modified.iloc[:,:5].describe()

,DR_1,|DR|_1,EMAi0.001_1,EMAi0.01_1
count,13892.000000,13891.000000,13896.000000,13889.000000
mean,50464.654067,6.635809,12.933389,18.736984
std,70250.969065,13.514586,17.700624,24.983520
min,-34751.069362,-10.279164,-7.194081,-8.547930
25%,6837.827239,2.287159,1.707703,2.566592
50%,21062.623620,3.969886,5.302680,7.753833
75%,63680.218975,8.607217,17.173314,25.908939
max,694227.459630,1334.313100,172.668025,225.630056


`describe()` bietet zusätzliche Informationen zu den jeweiligen Features. Dazu gehört die Anzahl, der Mittelwert (mean), die Standardabweichung (std) sowie das minimale, 25%, 50%, 75% und das maximale Quantil.

In [ ]:
df_modified['target'].value_counts()

Aceton high           1834
Toluol low            1425
Ethen low             1190
Ammoniak high         1156
Ethanol low           1083
Ethen high            1082
Ethanol high           963
Acetaldehyd high       836
Ethen medium           654
Aceton low             642
Acetaldehyd low        565
Acetaldehyd medium     535
Aceton medium          533
Ethanol medium         519
Toluol medium          397
Ammoniak medium        311
Ammoniak low           174
Toluol high             11
Name: target, dtype: int64

Wir haben `value_counts()` an der `target` Spalte aufgerufen und sehen das Aceton medium mit 1924 Einträge über die meisten Messungen verfügt. Gefolgt von Toluol low und so weiter.

### Pandas Profiling

Im folgenden soll noch Pandas Profiling vorgestellt werden. Pandas Profiling bietet ähnlich wie die `describe()` Methode statistische Auswertungen. Mehr dazu kann auf https://github.com/pandas-profiling/pandas-profiling gelesen werden. <br>

Der folgende Code (auskommentiiert da dieser Schritt zeitintensiv ist) erstellt ein vollständigen Bericht und speichert diesen als `report.html` im lokalen Verzeichnis ab.

In [ ]:
#profile = df_modified.profile_report(title='Pandas Profiling Report')
#profile.to_file(output_file="report.html")